In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
name = "FirstProject"
conf = SparkConf().setAppName(name).setMaster('local[1]')
sc = SparkContext(conf=conf)

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [48]:
#Создаем два RDD на основе таблиц и выводим их титульники
tripD = sc.textFile("/mnt/data/trip.csv")
stationD = sc.textFile("/mnt/data/station.csv")

In [50]:
tripHeader=tripD.first()
stationHeader=stationD.first()

In [51]:
#избавляемся от первой строки с заглавиями
trips= tripD.filter(lambda row: row !=tripHeader).map(lambda row: row.split(","))
stations=stationD.filter(lambda row: row !=stationHeader).map(lambda row: row.split(","))

In [52]:
trips.take(2)

[['4576',
  '63',
  '8/29/2013 14:13',
  'South Van Ness at Market',
  '66',
  '8/29/2013 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127'],
 ['4607',
  '70',
  '8/29/2013 14:42',
  'San Jose City Hall',
  '10',
  '8/29/2013 14:43',
  'San Jose City Hall',
  '10',
  '661',
  'Subscriber',
  '95138']]

In [10]:
stations.take(2)

[['2',
  'San Jose Diridon Caltrain Station',
  '37.329732',
  '-121.90178200000001',
  '27',
  'San Jose',
  '8/6/2013'],
 ['3',
  'San Jose Civic Center',
  '37.330698',
  '-121.888979',
  '15',
  'San Jose',
  '8/5/2013']]

In [11]:
#Ниже, после объединения получались нули, поэтому я решила попробовать 
#проиндексировать не по номеру велостоянки, а так же, по её названию
stationsIndexed= stations.keyBy(lambda row: (row[1]))
stationsIndexed.take(2)

[('San Jose Diridon Caltrain Station',
  ['2',
   'San Jose Diridon Caltrain Station',
   '37.329732',
   '-121.90178200000001',
   '27',
   'San Jose',
   '8/6/2013']),
 ('San Jose Civic Center',
  ['3',
   'San Jose Civic Center',
   '37.330698',
   '-121.888979',
   '15',
   'San Jose',
   '8/5/2013'])]

In [12]:
#Индексируем по начальной и конечной станциям
tripsStart = trips.keyBy(lambda row: row[3])
tripsEnd = trips.keyBy(lambda row: row[6])
tripsStart.take(2), tripsEnd.take(2)

([('South Van Ness at Market',
   ['4576',
    '63',
    '8/29/2013 14:13',
    'South Van Ness at Market',
    '66',
    '8/29/2013 14:14',
    'South Van Ness at Market',
    '66',
    '520',
    'Subscriber',
    '94127']),
  ('San Jose City Hall',
   ['4607',
    '70',
    '8/29/2013 14:42',
    'San Jose City Hall',
    '10',
    '8/29/2013 14:43',
    'San Jose City Hall',
    '10',
    '661',
    'Subscriber',
    '95138'])],
 [('South Van Ness at Market',
   ['4576',
    '63',
    '8/29/2013 14:13',
    'South Van Ness at Market',
    '66',
    '8/29/2013 14:14',
    'South Van Ness at Market',
    '66',
    '520',
    'Subscriber',
    '94127']),
  ('San Jose City Hall',
   ['4607',
    '70',
    '8/29/2013 14:42',
    'San Jose City Hall',
    '10',
    '8/29/2013 14:43',
    'San Jose City Hall',
    '10',
    '661',
    'Subscriber',
    '95138'])])

In [13]:
#объединяем по ключу
startTrips = stationsIndexed.join(tripsStart)
endTrips = stationsIndexed.join(tripsEnd)

In [14]:
#выводим полученные графы
startTrips.toDebugString()

b'(4) PythonRDD[31] at RDD at PythonRDD.scala:53 []\n |  MapPartitionsRDD[23] at mapPartitions at PythonRDD.scala:145 []\n |  ShuffledRDD[22] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(4) PairwiseRDD[21] at join at <ipython-input-13-9f06234f33cb>:2 []\n    |  PythonRDD[20] at join at <ipython-input-13-9f06234f33cb>:2 []\n    |  UnionRDD[19] at union at NativeMethodAccessorImpl.java:0 []\n    |  PythonRDD[17] at RDD at PythonRDD.scala:53 []\n    |  /mnt/data/station.csv MapPartitionsRDD[8] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  /mnt/data/station.csv HadoopRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  PythonRDD[18] at RDD at PythonRDD.scala:53 []\n    |  /mnt/data/trip.csv MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  /mnt/data/trip.csv HadoopRDD[5] at textFile at NativeMethodAccessorImpl.java:0 []'

In [15]:
endTrips.toDebugString()

b'(4) PythonRDD[32] at RDD at PythonRDD.scala:53 []\n |  MapPartitionsRDD[30] at mapPartitions at PythonRDD.scala:145 []\n |  ShuffledRDD[29] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(4) PairwiseRDD[28] at join at <ipython-input-13-9f06234f33cb>:3 []\n    |  PythonRDD[27] at join at <ipython-input-13-9f06234f33cb>:3 []\n    |  UnionRDD[26] at union at NativeMethodAccessorImpl.java:0 []\n    |  PythonRDD[24] at RDD at PythonRDD.scala:53 []\n    |  /mnt/data/station.csv MapPartitionsRDD[8] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  /mnt/data/station.csv HadoopRDD[7] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  PythonRDD[25] at RDD at PythonRDD.scala:53 []\n    |  /mnt/data/trip.csv MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  /mnt/data/trip.csv HadoopRDD[5] at textFile at NativeMethodAccessorImpl.java:0 []'

In [16]:
startTrips.count(),endTrips.count()

(658617, 657435)

### Объявление классов

In [17]:
#Исправила объявление классов
from typing import NamedTuple
from datetime import datetime

def ToTrip(rows):
    class Trip(NamedTuple):
        tripId: int
        duration: int
        startDate: str
        startStation: str
        startTerminal: int
        endDate: str
        endStation: str
        endTerminal: int
        bikeId: int
        subscriptionType: str
        zipCode: str



    for row in rows:
        yield Trip(                             
             tripId=int(row[0]),
             duration=int(row[1]),
             startDate=datetime.strptime(row[2], '%m/%d/%Y %H:%M'),
             startStation=row[3],
             startTerminal=int(row[4]),
             endDate=datetime.strptime(row[5], '%m/%d/%Y %H:%M'),
             endStation=row[6],
             endTerminal=row[7],
             bikeId=int(row[8]),
             subscriptionType=row[9],
             zipCode=row[10]
        )



In [18]:
tripsInternal = trips.mapPartitions(ToTrip)

In [19]:
tripsInternal.take(2)

[Trip(tripId=4576, duration=63, startDate=datetime.datetime(2013, 8, 29, 14, 13), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 14, 14), endStation='South Van Ness at Market', endTerminal='66', bikeId=520, subscriptionType='Subscriber', zipCode='94127'),
 Trip(tripId=4607, duration=70, startDate=datetime.datetime(2013, 8, 29, 14, 42), startStation='San Jose City Hall', startTerminal=10, endDate=datetime.datetime(2013, 8, 29, 14, 43), endStation='San Jose City Hall', endTerminal='10', bikeId=661, subscriptionType='Subscriber', zipCode='95138')]

In [20]:
def ToStation(rows):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str
    

    for row in rows:
        yield Station(
            station_id=int(row[0]),
            name=row[1],
            lat=float(row[2]),
            long=float(row[3]),
            dockcount=int(row[4]),
            landmark=row[5],
            installation=datetime.strptime(row[6], '%m/%d/%Y'),  
        )

In [21]:
stationsInternal = stations.mapPartitions(ToStation)

In [22]:
stationsInternal.take(2)

[Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0)),
 Station(station_id=3, name='San Jose Civic Center', lat=37.330698, long=-121.888979, dockcount=15, landmark='San Jose', installation=datetime.datetime(2013, 8, 5, 0, 0))]

In [23]:
#считаем среднее время поездки
#сначала получаем коллекцию пар ключей
#Объединяем по начальной станции
tripsByStartStation = tripsInternal.keyBy(lambda record: record.startStation)

In [24]:
tripsByStartStation.first()

('South Van Ness at Market',
 Trip(tripId=4576, duration=63, startDate=datetime.datetime(2013, 8, 29, 14, 13), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 14, 14), endStation='South Van Ness at Market', endTerminal='66', bikeId=520, subscriptionType='Subscriber', zipCode='94127'))

In [25]:
from functools import reduce
#Рассчитываем среднее время поездки для каждого стартового парковочного места
avgDurationByStartStation = tripsByStartStation\
 .mapValues(lambda trip: trip.duration)\
 .groupByKey()\
 .mapValues(lambda values: reduce(lambda v1, v2: (v1 + v2) , values) / len(values))

In [26]:
avgDurationByStartStation.top(10, lambda x: x[1])

[('University and Emerson', 7090.239417989418),
 ('California Ave Caltrain Station', 4628.005847953216),
 ('Redwood City Public Library', 4579.234741784037),
 ('Park at Olive', 4438.1613333333335),
 ('San Jose Civic Center', 4208.016938519448),
 ('Rengstorff Avenue / California Street', 4174.082373782108),
 ('Redwood City Medical Center', 3959.491961414791),
 ('Palo Alto Caltrain Station', 3210.6489815253435),
 ('San Mateo County Center', 2716.7700348432054),
 ('Broadway at Main', 2481.2537313432836)]

In [27]:
#Это тоже самое делаем но с помощью другого метода
avgDurationByStartStation2 = tripsByStartStation.mapValues(lambda x:x.duration)\
  .aggregateByKey((0,0),lambda acc, value: (acc._1 + value, acc._2 + 1),
   lambda acc1, acc2: (acc1._1+acc2._1, acc1._2+acc2._2))\
  .mapValues(lambda acc:acc._1/acc._2)


In [28]:
avgDurationByStartStation.top(10, lambda x: x[1])

[('University and Emerson', 7090.239417989418),
 ('California Ave Caltrain Station', 4628.005847953216),
 ('Redwood City Public Library', 4579.234741784037),
 ('Park at Olive', 4438.1613333333335),
 ('San Jose Civic Center', 4208.016938519448),
 ('Rengstorff Avenue / California Street', 4174.082373782108),
 ('Redwood City Medical Center', 3959.491961414791),
 ('Palo Alto Caltrain Station', 3210.6489815253435),
 ('San Mateo County Center', 2716.7700348432054),
 ('Broadway at Main', 2481.2537313432836)]

In [29]:
#ищем первую поездку для каждой стоянки
firstGrouped  = tripsByStartStation\
    .reduceByKey(
        lambda trip1, trip2: trip1 if trip1.startDate < trip2.startDate else trip2)

In [30]:
firstGrouped.first()

('South Van Ness at Market',
 Trip(tripId=4074, duration=1131, startDate=datetime.datetime(2013, 8, 29, 9, 24), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 9, 43), endStation='San Francisco Caltrain 2 (330 Townsend)', endTerminal='69', bikeId=317, subscriptionType='Subscriber', zipCode='94115'))

### Сама лабораторная работа

#### Ищем велоcипед с максимальным пробегом

In [32]:
#группируем по id
Bike = tripsInternal.keyBy(lambda trip: trip.bikeId)
Bike.first()

(520,
 Trip(tripId=4576, duration=63, startDate=datetime.datetime(2013, 8, 29, 14, 13), startStation='South Van Ness at Market', startTerminal=66, endDate=datetime.datetime(2013, 8, 29, 14, 14), endStation='South Van Ness at Market', endTerminal='66', bikeId=520, subscriptionType='Subscriber', zipCode='94127'))

In [33]:
#потом получаем продолжительности и скадываем их для каждой поездки
BikeDuration = Bike\
    .mapValues(lambda trip: trip.duration)\
    .reduceByKey(lambda trip1, trip2: trip1 + trip2)


In [34]:
#Велоcипед с максимальным пробегом
BikeDuration = BikeDuration.top(1, key=lambda x: x[1])[0][0]
BikeDuration

535

#### Найти наибольшее расстояние между станциями.

In [36]:
#Исправленное
#Метод для вычисления расстояний между двумя станциями
import math 
from math import radians 
earthRadiusKm=6371.0088

def distanceEarth( lat1d,  lon1d,  lat2d,  lon2d):
    lat1r = radians(lat1d)
    lon1r = radians(lon1d)
    lat2r = radians(lat2d)
    lon2r = radians(lon2d)
    u = math.sin((lat2r - lat1r)/2)
    v = math.sin((lon2r - lon1r)/2)
    d = u ** 2 + math.cos(lat1r) * math.cos(lat2r) *(v ** 2)
    return 2.0 * earthRadiusKm *\
math.asin(math.sqrt(u ** 2 + math.cos(lat1r) * math.cos(lat2r) *(v ** 2)))


In [38]:
Trips2= stationsInternal.cartesian(stationsInternal)\
    .filter(lambda x: x[0].station_id != x[1].station_id)\
    .map(lambda s: (s[0], s[1], distanceEarth(s[0].lat, s[0].long,s[1].lat, s[1].long)))

In [39]:
#Выводим станции с наибольшим расстоянием между ними
Trips2.top(1, key=lambda x: x[2])

[(Station(station_id=16, name='SJSU - San Salvador at 9th', lat=37.333954999999996, long=-121.877349, dockcount=15, landmark='San Jose', installation=datetime.datetime(2013, 8, 7, 0, 0)),
  Station(station_id=60, name='Embarcadero at Sansome', lat=37.80477, long=-122.40323400000001, dockcount=15, landmark='San Francisco', installation=datetime.datetime(2013, 8, 21, 0, 0)),
  69.92097253310907)]

#### Найти путь велосипеда с максимальным пробегом через станции.

In [40]:
MaxDuration = tripsInternal.filter(lambda trip: trip.bikeId == BikeDuration)\
    .sortBy(lambda trip: trip.startDate).map(lambda trip:trip.startStation)    

In [41]:
#Выводим последовательность станций по которым он проехал
MaxDuration.collect()

['Post at Kearney',
 'San Francisco Caltrain (Townsend at 4th)',
 'San Francisco Caltrain 2 (330 Townsend)',
 'Market at Sansome',
 '2nd at Townsend',
 'San Francisco City Hall',
 'Civic Center BART (7th at Market)',
 'Post at Kearney',
 'Embarcadero at Sansome',
 'Washington at Kearney',
 'Market at Sansome',
 'Market at Sansome',
 '2nd at Folsom',
 'Temporary Transbay Terminal (Howard at Beale)',
 '2nd at Townsend',
 'Embarcadero at Sansome',
 'Clay at Battery',
 'Harry Bridges Plaza (Ferry Building)',
 'Clay at Battery',
 'San Francisco Caltrain (Townsend at 4th)',
 'Steuart at Market',
 '2nd at Townsend',
 'Harry Bridges Plaza (Ferry Building)',
 'Townsend at 7th',
 'San Francisco Caltrain 2 (330 Townsend)',
 'San Francisco Caltrain 2 (330 Townsend)',
 'Steuart at Market',
 'San Francisco Caltrain (Townsend at 4th)',
 '2nd at South Park',
 'Post at Kearney',
 '2nd at Folsom',
 'Mechanics Plaza (Market at Battery)',
 'Powell at Post (Union Square)',
 'Powell at Post (Union Square)',

#### Найти количество велосипедов в системе

In [42]:
number_bicycles = tripsInternal.keyBy(lambda r: r.bikeId).groupByKey()

In [43]:
number_bicycles.count()

700

#### Найти пользователей потративших на поездки более 3 часов.

In [44]:
#Теперь в секундах
#И идентифицировать пользователей, как я поняла, можно по zip_code

In [45]:
#Так как zip_code не является обязательным, то ещё чистим пустые
#И удаляем одинаковые
Id_trip = tripsInternal.filter(lambda trip: trip.duration >180*60)\
    .map(lambda trip: trip.zipCode).filter(lambda trip: trip!='').distinct()

In [46]:
Id_trip.take(20)

['58553',
 '94301',
 '94039',
 '94133',
 '93726',
 '94123',
 '4517',
 '29200',
 '45322',
 '94080',
 '92808',
 '5024',
 '89138',
 '11515',
 '28277',
 '34990',
 '94803',
 '92663',
 '94109',
 '91801']

In [47]:
Id_trip.count()

2100